In [ ]:
#imports all the needed packages
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
import time
import numpy as np
import pandas as pd
from sqlalchemy import *
import threading
from datetime import datetime
import networkx
from odo import odo

In [ ]:
#defines the functions that are used later
def get_html(browser):
    form = browser.find_elements_by_css_selector('#main > div > div:nth-child(3n)')
    html = form[0].get_attribute('innerHTML')
    bs = BeautifulSoup(html, 'html.parser')
    cups = bs("table")[0].findAll("tbody")
    return cups
def parse_the_table(cups):
    #create an empty table
    int_time = datetime.now()
    t_names = ['date', 'time', 'cup', 'players', 'set', 'score0', 'score1', 'one1', 'two1', 'hcap1', 'one2', 'hcap2', \
                'two2', 'total', 'o', 'u', 'extras']
    table = pd.DataFrame(columns = t_names)
    #fill in the table
    cnt = 0
    for b in range(len(cups)):
        rows = cups[b].findAll('tr')
        cols0 = rows[0].findAll('th')
        colnames = t_names[6:len(t_names)]
        cup_name = cols0[0].findAll('span', {"class":"table__title-text"})[0].string #! ('span)[1] for tennis and ('span)[2] for table-tennis
        for r in range(1, len(rows)):
            l = len(rows[r].findAll('td')[1].findAll('a'))
            table.loc[cnt, 'date'] = int_time.date().strftime('%Y-%m-%d %H:%M:%S')
            table.loc[cnt, 'time'] = int_time.time().strftime('%Y-%m-%d %H:%M:%S')
            table.loc[cnt, 'cup'] = cup_name
            if l != 0:
                cols1 = rows[r].findAll('td')
                players = cols1[1].findAll('div')[0]('a')[0].string #contains the string with the players
                table.loc[cnt, 'players'] = players
                table.loc[cnt, 'set'] = None
                score_list = cols1[1].findAll('div')[1].findAll('div',{"class":"table__score"})[0].findAll("span")
                table.loc[cnt, 'score0'] = score_list[0].string
                table.loc[cnt, 'score1'] = score_list[1].string #might be different for tennis
                for c in range(2, len(cols1)):
                    table.loc[cnt, colnames[c-1]] = cols1[c].string
            else:
                table.loc[cnt, 'players'] = None
                cols2 = rows[r].findAll('td')
                set_num = cols2[1].findAll('div')[0].contents[2]  #contains the set's number
                table.loc[cnt, 'set'] = set_num
                ###set_score = cols2[1].findAll('div')[1].span.string #contains the set score
                score_list = cols2[1].findAll('div')[1].findAll("span")
                if len(score_list) == 1:       #---Base case version
                    table.loc[cnt, 'score0'] = score_list[0].string
                    table.loc[cnt, 'score1'] = None
                elif len(score_list) == 2:
                    table.loc[cnt, 'score0'] = score_list[0].string
                    table.loc[cnt, 'score1'] = score_list[1].string
                else:
                    table.loc[cnt, 'score0'] = None
                    table.loc[cnt, 'score1'] = None
                for c in range(2, len(cols2)):
                    table.loc[cnt, colnames[c-1]] = cols2[c].string
            cnt += 1
    return table
def connect_to_google_sql(conn_string):
    try:
        engine = create_engine(conn_string, echo=True)
        conn = engine.connect()
        return [engine, conn]
    except:
        print('An error occurred!')
        return [None]
def close_google_cloud_conn(cn):
    if len(cn) == 2:
        cn[1].close()
        cn[0].dispose()
def get_table(browser):
    x0 = time.time()
    cups = get_html(browser)
    table = parse_the_table(cups)
    return table
def init_container():
    t_names = ['date', 'time', 'cup', 'players', 'set', 'score0', 'score1', 'one1', 'two1', 'hcap1', 'one2', 'hcap2', \
                    'two2', 'total', 'o', 'u', 'extras']
    container = pd.DataFrame(columns = t_names)
    return container
def update_container(container, table):
    arr = [container, table]
    new_cont = pd.concat(arr)
    return new_cont

In [ ]:
#launches the Chrome driver
browser = webdriver.Chrome('path/to/chromedriver.exe')
ref = ''
browser.get(ref)

In [ ]:
#checks if you can get the information table
table = get_table(browser)
table

In [ ]:
#collects the info tables every second and stores them in variable "container"
iterations = 60 * 60 * 2
container = init_container()
for i in range(iterations):
    try:
        table = get_table(browser)
        time.sleep(0.9)
        container = update_container(container, table)
        print('Iteration: ' + str(i) + '. ' + str(len(table)) + ' rows inserted.')
    except:
        print('Some error occurred on iteration ' + str(i))

In [ ]:
#sends the data to Google Cloud SQL
t0 = time.time()
t = odo(container, 'sql_path_string')
print(str(len(container)) + ' rows inserted. Time elapsed: ' + str(time.time() - t0))

In [ ]:
#or just saves it as a .xlsx file
container.to_excel('path/to/where/you/want/to/store/it/container.xlsx')